In [1]:
# load package
import numpy as np
import pandas as pd
import jieba
from collections import Counter


# set path
import os
default_path = "/Users/francislin/jrml/What's Cooking/"
os.chdir(default_path)

# skip interaction warning
import warnings; 
warnings.filterwarnings('ignore')
#你可以选择修改 ast_note_iteractively kernel 选项来使得 Jupyter 为每一行的变量或语句执行这个操作，以便你可以立即看到多条语句一起输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
train=pd.read_json('./data/train.json')
test=pd.read_json('./data/test.json')
submission=pd.read_csv('./data/sample_submission.csv')

In [3]:
train.head()

,cuisine,id,ingredients
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes..."
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g..."
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,indian,22213,"[water, vegetable oil, wheat, salt]"
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe..."


In [4]:
test.head()

,id,ingredients
0,18009,"[baking powder, eggs, all-purpose flour, raisi..."
1,28583,"[sugar, egg yolks, corn starch, cream of tarta..."
2,41580,"[sausage links, fennel bulb, fronds, olive oil..."
3,29752,"[meat cuts, file powder, smoked sausage, okra,..."
4,35687,"[ground black pepper, salt, sausage casings, l..."


In [5]:
df=train.append(test, ignore_index=True)
del (train, test)

In [6]:
content = list(df['ingredients'])
contenttxt = ''.join(str(e) for e in content)
#把全部要分析的ingredients灌進一個string

In [7]:
#把結巴沒斷好的拿掉
garbage = ['teeeest']

In [18]:
# 計算詞頻
def get_words(txt, num=900):
    seg_list = jieba.cut(txt, cut_all=False)
    temp = list(seg_list)
    temp = [x.strip() for x in temp]
    temp = [x for x in temp if x not in garbage]
    c = Counter()
    for x in temp:
        if len(x)>1 and x != '\r\n':
            c[x] += 1
    return c.most_common(num)

In [19]:
frq = get_words(contenttxt)
print(frq)
print(type(frq))

[('pepper', 31909), ('salt', 30573), ('oil', 29048), ('garlic', 23614), ('ground', 22836), ('fresh', 22161), ('sauce', 15987), ('sugar', 15621), ('onions', 15450), ('cheese', 14638), ('chicken', 14300), ('olive', 13539), ('black', 13465), ('water', 12219), ('red', 11525), ('flour', 11082), ('butter', 10804), ('tomatoes', 10717), ('green', 10643), ('powder', 10302), ('chopped', 9802), ('cloves', 9422), ('juice', 8970), ('white', 8626), ('onion', 8609), ('eggs', 8170), ('rice', 7661), ('cream', 7628), ('cilantro', 7432), ('milk', 7353), ('lemon', 7288), ('vegetable', 7068), ('leaves', 6837), ('large', 6787), ('ginger', 6732), ('corn', 6672), ('dried', 6647), ('lime', 6199), ('vinegar', 6185), ('soy', 6076), ('all', 6054), ('purpose', 6045), ('cumin', 5809), ('broth', 5556), ('chili', 5406), ('wine', 5334), ('bell', 5313), ('parsley', 5172), ('sesame', 4391), ('beans', 4353), ('grated', 4093), ('kosher', 4012), ('carrots', 4005), ('extra', 3856), ('basil', 3835), ('beef', 3798), ('dry', 3

In [20]:
#計算出用字詞頻，以利未來製作feature標籤
df_frq = pd.DataFrame(frq, columns=["words","frq"])
df_frq

,words,frq
0,pepper,31909
1,salt,30573
2,oil,29048
3,garlic,23614
4,ground,22836
5,fresh,22161
6,sauce,15987
7,sugar,15621
8,onions,15450
9,cheese,14638


In [21]:
df_frq.to_csv("df_frq.csv", index= False)

In [22]:
def list_2_str(L):
    str1 = ''.join(L)
    return str1

In [23]:
df['ingred']= df['ingredients'].apply(lambda x: list_2_str(x))

In [24]:
#用結巴出來的結果系統性貼標
for a in df_frq['words']:
    df['used_%s'% (a)]= df['ingred'].str.contains('%s'% (a),na=False)*1
    df['used_%s'% (a)]= np.int8(df['used_%s'% (a)])

In [25]:
df.head()

,cuisine,id,ingredients,ingred,used_pepper,used_salt,used_oil,used_garlic,used_ground,used_fresh,...,used_hock,used_rose,used_ciabatta,used_ready,used_regular,used_glass,used_safflower,used_jeera,used_Oil,used_oven
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes...",romaine lettuceblack olivesgrape tomatoesgarli...,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g...",plain flourground peppersalttomatoesground bla...,1,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g...",eggspeppersaltmayonaisecooking oilgreen chilie...,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,indian,22213,"[water, vegetable oil, wheat, salt]",watervegetable oilwheatsalt,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe...",black peppershallotscornflourcayenne pepperoni...,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
del (df_frq, frq, contenttxt, content)

In [28]:
df=df.drop(['ingredients','ingred'],axis=1)

In [29]:
os.chdir('/Users/francislin/temp/')
import pickle
with open('all_data.pickle','wb') as f:
    pickle.dump(df,f)

In [30]:
%pwd

'/Users/francislin/temp'

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49718 entries, 0 to 49717
Columns: 904 entries, cuisine to used_oven
dtypes: int64(1), int8(900), object(3)
memory usage: 44.2+ MB
